# Importar librerias

In [21]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from quickda.clean_data import *

# Limpiar los datos y seleccionar las columnas a utilizar

In [22]:
datos = pd.read_csv('baseball_reference_2016_scrape.csv')

datos["attendance"] = datos["attendance"].str.replace("']", "", regex=True)
datos["attendance"] = datos["attendance"].str.replace(",", "", regex=True)
datos = datos[datos["attendance"].str.isdigit()]
datos = datos.astype({"attendance": int})

datos['game_duration'] = datos['game_duration'].str.replace(": ", "")
v = datos["game_duration"].str.split(':', expand=True).astype(int)
datos["game_duration"] = v[0]*60 +v[1]
datos = datos.astype({"attendance": int}, {'game_duration': int})

datos["venue"] = datos["venue"].str.replace(": ", "", regex=True)

datos = datos[["attendance", "home_team", "away_team", "game_duration", "start_time", "venue"]]

# Codificar los datos

In [24]:
X = datos.iloc[:, 1:]
y = datos.iloc[:, 0]

variables_a_codificar = ["home_team", "away_team", "venue", "start_time", "game_duration"]
transCol = ColumnTransformer(transformers=[('encoder', OneHotEncoder(sparse_output=False), variables_a_codificar)], remainder='passthrough')
X = np.array(transCol.fit_transform(X))

# Crear el modelo

In [25]:
X_entreno, X_prueba, y_entreno, y_prueba = train_test_split(X, y, test_size = 0.2, random_state = 0)
regresor = LinearRegression()
regresor.fit(X_entreno, y_entreno)

# Predecir un resultado

In [27]:
varToPredict = []
    
homeTeam = "Kansas City Royals"
awayTeam = "New York Mets"
gameDuration = "193"
startTime = "Start Time: 7:38 p.m. Local"
venue = "Kauffman Stadium"

for column in transCol.get_feature_names_out():
    splitVal = column.replace("encoder__", "").split("_")
    valType = splitVal[0]
    testVal = splitVal[len(splitVal) - 1].strip()

    if venue == testVal and valType == "venue":
        varToPredict.append(1)
    elif homeTeam == testVal and valType == "home":
        varToPredict.append(1)
    elif awayTeam == testVal and valType == "away":
        varToPredict.append(1)
    elif gameDuration == testVal and valType == "game":
        varToPredict.append(1)
    elif startTime == testVal and valType == "start":
        varToPredict.append(1)
    else:
        varToPredict.append(0)

print(regresor.predict([varToPredict]))

[43040.]


# Cálculo de R cuadrado

In [28]:
r_squared = regresor.score(X_entreno, y_entreno)
print(r_squared)

0.7454042906456203
